In [2]:
import os
from langchain.document_loaders import PyPDFLoader

os.environ["OPENAI_API_KEY"] = "<your-openai-key>"

In [3]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# Prevent hallucination with prompt engineering
# load the document as before
loader = PyPDFLoader('./docs/workation_1.pdf')
documents = loader.load()

# we split the data into chunks of 1,000 characters, with an overlap
# of 200 characters between the chunks, which helps to give better results
# and contain the context of the information between chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(documents)

# we create our vectorDB, using the OpenAIEmbeddings tranformer to create
# embeddings from our text chunks. We set all the db information to be stored
# inside the ./data directory, so it doesn't clutter up our source files
vectordb = Chroma.from_documents(
  documents,
  embedding=OpenAIEmbeddings(),
  # persist_directory='./data'
)

In [4]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

qa_chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(temperature=1, model_name="gpt-3.5-turbo"),
    vectordb.as_retriever(search_kwargs={'k': 4}),
)

In [83]:
import pandas as pd
import numpy as np
import pickle
import random

# Load data (-> DataFrame)

# f = open("docs/workation_accommodation.csv", "r", encoding='cp949')
# text = f.readlines()
# print(text)

# ----------------------------------------------------------------------
work_acco = pd.read_csv("docs/workation_accommodation.csv", encoding='cp949')
work_rest = pd.read_csv("docs/workation_restaurant.csv", encoding='cp949')
work_tour = pd.read_csv("docs/workation_tourism.csv", encoding='utf8')
# ----------------------------------------------------------------------




In [123]:
import datetime

startDate = '03/08/2023'
endDate = '05/08/2023'

format_str = '%d/%m/%Y'  

startDate_obj = datetime.datetime.strptime(startDate, format_str) 
endDate_obj = datetime.datetime.strptime(endDate, format_str) 

days = int(endDate_obj.strftime("%d")) - int(startDate_obj.strftime("%d")) + 1

print(days)

3


In [125]:

# Accommodation Filterting
# ----------------------------------------------------------------------
selectedRegion = ["제주시", "제주 시내"]
#wholeTags = ["도심형", "소도시형", "해양형", "산악형", "스테이형", "호텔형", "워케이션 특화형"]
selectedTag = ["도심형",  "해양형", "산악형"]

mask_1 = (work_acco["행정구역"] == selectedRegion[0]) & (work_acco["지역"] == selectedRegion[1])
mask_2 = (work_acco["카테고리_1"].isin(selectedTag)) | (work_acco["카테고리_2"].isin(selectedTag))

user_acco = work_acco.loc[mask_1 & mask_2, :]

seleectedIndex = random.choice(user_acco.index.tolist())
recommend_acco = user_acco.loc[seleectedIndex, :]

# ----------------------------------------------------------------------

# Tourism Filterting
# ----------------------------------------------------------------------
selectedRegion = ["제주시", "제주시내"]
# wholeTag = ["골프", "다이빙", "서핑", "승마", "역사유적", "오름", "요가", "이색체험", "자연경관", "전시/미술관", "카트/ATV", "테마파크", "해변", "카페", "보트"]
selectedTag = ["다이빙", "서핑", "역사유적", "이색체험", "카트/ATV", "카페", "보트", "역사유적", "오름", "요가", "전시/미술관", "카트/ATV", ]
# selectedTag = ["골프", "다이빙", "서핑", "승마",  "이색체험", "자연경관", "전시/미술관", "카트/ATV", "테마파크", "해변", "카페", "보트"]


mask_1 = (work_tour["행정구역"] == selectedRegion[0]) & (work_tour["지역"] == selectedRegion[1])
mask_2 = (work_tour["태그"].isin(selectedTag))

user_tour = work_tour.loc[mask_1 & mask_2, :]

# seleectedIndex = random.choice(user_tour.index.tolist())
# recommend_tour  = work_tour.loc[seleectedIndex, :]
# ----------------------------------------------------------------------

# Restaurant Filterting
# ----------------------------------------------------------------------

selectedRegion = ["제주시", "제주시내"]

mask_1 = (work_rest["행정구역"] == selectedRegion[0]) & (work_rest["지역"] == selectedRegion[1])

user_rest = work_rest.loc[mask_1, :]

# seleectedIndex = random.choice(user_tour.index.tolist())
# recommend_tour  = work_tour.loc[seleectedIndex, :]

# ----------------------------------------------------------------------

In [126]:
user_acco

,대분류,카테고리_1,카테고리_2,워케이션 이름,행정구역,지역,주소,소개,예약링크,전화번호
0,워케이션 숙소,스테이형,도심형,고요산책,제주시,제주 시내,제주시 중앙로 12길 5 (1층),"더 나은 내일, 나를 고민하는 여행자를 위한 조용한 북스테이\n1층 북라운지와 2~...",www.instagram.com/goyowalk_jeju,0507-1330-2036


In [127]:
user_rest

,대분류,식당 이름,주소,행정구역,지역,전화번호,예약링크
178,맛집,별오름 노형점,제주 제주시 1100로 3213-27 B201호 (노형동),제주시,제주시내,NaN,NaN
179,맛집,그라나다,제주 제주시 1100로 3308 1층 (노형동),제주시,제주시내,NaN,NaN
180,맛집,고로케야,제주 제주시 가령골2길 13-20 (이도2동),제주시,제주시내,NaN,NaN
181,맛집,알드레항아리수제비,제주 제주시 가령로 19 지하 (이도이동),제주시,제주시내,NaN,NaN
182,맛집,한라산조랑말,제주 제주시 가령로 5 (이도이동),제주시,제주시내,NaN,NaN
...,...,...,...,...,...,...,...
753,맛집,수너베러,제주특별자치도 제주시 천수동로2길 29 (일도2동),제주시,제주시내,NaN,NaN
754,맛집,카고크루즈,제주특별자치도 제주시 탑동로 43 2층 (삼도이동),제주시,제주시내,NaN,NaN
755,맛집,지오포카치아,제주특별자치도 제주시 테우해안로 136 1층 (이호일동),제주시,제주시내,NaN,NaN
756,맛집,굽써,제주특별자치도 제주시 테우해안로 138 1층 (이호일동),제주시,제주시내,NaN,NaN


In [128]:
user_tour

,대분류,관광지 이름,행정구역,지역,주소,태그,소개,위도,경도,전화번호
3,관광지,제주소리,제주시,제주시내,제주특별자치도 제주시 신대로10길 55,이색체험,제주 민요에 녹아있는 제주 사람들의 이야기,33.490820,126.490681,010-9087-2953
18,관광지,제주농업학교 옛터,제주시,제주시내,제주특별자치도 제주시 전농로 100,역사유적,4?3이 발발하면서 수많은 제주도민들이 잡혀와 고문과 취조를 당했던 한과 눈물의 장소,33.499622,126.531190,NaN
19,관광지,산지항 옛터,제주시,제주시내,제주특별자치도 제주시 연무정길 28 (건입동)일대,역사유적,1950년 예비검속 당시 에비검속자 가운데 일부가 수장당한 곳.,33.516888,126.534690,NaN
20,관광지,화북환해장성,제주시,제주시내,제주특별자치도 제주시 화북동,역사유적,오랫동안 제주를 지켜주던 성벽에서 찾는 역사의 흔적,33.520233,126.565530,NaN
21,관광지,4·3해원방사탑,제주시,제주시내,제주특별자치도 제주시 신산로 82,역사유적,다시는 이 땅에 4?3과 같은 비극이 재발되지 않도록 돌을 쌓아 만든 방사탑,33.505470,126.534510,NaN
22,관광지,삼사석,제주시,제주시내,제주특별자치도 제주시 화북1동 1380-1,역사유적,제주 삼신인(고.양.부)이 쏘았던 화살이 꽂혔던 돌멩이를 보관한 곳,33.521027,126.574100,064-728-2735
30,관광지,거친오름,제주시,제주시내,제주특별자치도 제주시 명림로 520 (봉개동),오름,노루가 뛰어노는 거친오름,33.445274,126.619450,064-728-3611
32,관광지,밸런싱 요가,제주시,제주시내,제주특별자치도 제주시 중앙로 492,요가,잔잔한 호흡의 요가와 함께 기구를 활용한 필라테스를 즐겨보자. 제주 시내 아라동에 ...,33.479192,126.543192,0507-1401-6542
36,관광지,제주소리,제주시,제주시내,제주특별자치도 제주시 신대로10길 55,이색체험,제주 민요에 녹아있는 제주 사람들의 이야기,33.490820,126.490681,010-9087-2953
39,관광지,갤러리2 중선농원,제주시,제주시내,제주특별자치도 제주시 영평길 269,전시/미술관,감귤농장을 탈바꿈한 갤러리,33.478135,126.570170,064-755-2112


In [174]:
recommend_rest_list = []
recommend_tour_list = []

f = open("chat_query.txt", "w", encoding="cp949")

f.write("[양식]\n")
f.write("==============================================================================================================\n")
f.write(startDate_obj.strftime("%y-%m-%d") + "~" + endDate_obj.strftime("%y-%m-%d") + "까지의 워케이션 일정\n")
f.write("--------------------------------------------------------------------------------------------------------------\n")

f.write(f'워케이션 장소 : {recommend_acco["워케이션 이름"]}\n')
f.write(f'워케이션 종류 : {recommend_acco["카테고리_1"]}, {recommend_acco["카테고리_2"]}\n')
f.write(f'주소 : {recommend_acco["주소"]}\n')
f.write(f'전화번호 : {recommend_acco["주소"]}\n')
f.write(f'정보 : {recommend_acco["소개"]}\n')
f.write(f'예약하기 : {recommend_acco["예약링크"]}\n\n')

for i in range(days):
    # 숙박 장소
    f.write(f"[{i+1}번째 날]\n\n")
    
    # 추천된 맛집 장소
    # user_rest

    for _ in range(2):
        seleectedRestIndex = random.choice(list(set(user_rest.index.tolist()) - set(recommend_rest_list)))
        recommend_rest_list.append(seleectedRestIndex)
        recommend_rest = work_rest.loc[seleectedRestIndex, :]

        f.write(f'추천된 맛집 : {recommend_rest["식당 이름"]}\n')
        f.write(f'주소 : {recommend_rest["주소"]}\n')

    for _ in range(2):
        seleectedTourIndex = random.choice(list(set(user_tour.index.tolist()) - set(recommend_tour_list)))
        recommend_tour_list.append(seleectedTourIndex)
        recommend_tour = work_tour.loc[seleectedTourIndex, :]

        f.write(f'추천된 관광지 : {recommend_tour["관광지 이름"]}\n')
        f.write(f'종류 : {recommend_tour["태그"]}\n')
        f.write(f'주소 : {recommend_tour["주소"]}\n')
        f.write(f'전화번호 : {recommend_tour["전화번호"]}\n')
        f.write(f'소개 : {recommend_tour["소개"]}\n\n\n')


f.write("--------------------------------------------------------------------------------------------------------------\n")
f.write("==============================================================================================================\n")
f.write("위 계획에 대해서 친근하게 알려주는 워케이션 계획표를 작성해줘. 모르다면 모른다고 해줘. \n")

f.close()

In [176]:
f = open("chat_query.txt", "r")

chat_query = f.read()

In [178]:
import sys

chat_history = list()

while True:
    result = qa_chain({'question': chat_query, 'chat_history': chat_history})

    print("Application Answer: ", result["answer"])

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 4305 tokens. Please reduce the length of the messages.